In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Краткое описание

Ноутбук предназначен для того, чтобы делать выборку кандидатов по ключевым словам

# Resume samples

In [ ]:
#from parse_hh_data import download, parse
#import requests
import time
import pandas as pd
import json
import random as rd
import numpy as np
from sklearn.metrics import accuracy_score

## Working with dataset

In [ ]:
with open('drive/MyDrive/LD final/data_file_it.json') as f:
    data = json.load(f)

In [ ]:
data[1]['education'][0]['year']

2011

In [ ]:
gender = []
area = []
spec_name = [] #['specialization'][0]['name']
profarea_name = []#['specialization'][1]['profarea_name']
title = [] #['title']
salary = [] #['salary']['amount']
edu = [] #['education_level']
exp = [] #['experience'][i]['position']
skill_set = [] #['skill_set']
skills = [] #['skills']
birth_date = [] 
first_educ_year = []

for resume in data:
  print(resume)
  gender.append(resume['gender'])
  area.append(resume['area'])
  title.append(resume['title'])
  salary.append(resume['salary']['amount'])
  edu.append(resume['education_level'])
  skill_set.append(resume['skill_set'])
  skills.append(resume['skills'])
  if resume['birth_date']!=None and resume['education']!=None:
    birth_date.append(resume['birth_date'][-4:])
    first_educ_year.append(resume['education'][0]['year'])

  exp_temp = []
  for i in range(len(resume['experience'])):
      exp_temp.append(resume['experience'][i]['position'])
  exp.append(exp_temp)
  spec_temp = []
  for i in range(len(resume['specialization'])):
      spec_temp.append(resume['specialization'][i]['name'])
  spec_name.append(spec_temp)
    
    
df = pd.DataFrame([gender,area,  title, salary, edu, exp, skill_set, skills]).transpose()


{'birth_date': '23\xa0августа\xa01994', 'gender': 'Мужчина', 'area': 'Москва', 'title': 'Специалист по машинному обучению', 'specialization': [{'name': 'Программирование, Разработка', 'profarea_name': 'Информационные технологии, интернет, телеком'}, {'name': 'Аналитик', 'profarea_name': 'Информационные технологии, интернет, телеком'}], 'salary': {'amount': 120000, 'currency': 'руб.'}, 'education_level': 'Высшее образование (Магистр)', 'education': [{'year': 2018, 'name': 'Московский государственный университет им. М.В. Ломоносова, Москва', 'organization': 'Физический, Нелинейная оптика и лазерная физика'}, {'year': 2016, 'name': 'Московский государственный университет им. М.В. Ломоносова, Москва', 'organization': 'Физический, Физика'}], 'language': [{'name': 'Русский', 'level': 'Родной'}, {'name': 'Английский', 'level': 'B1 - Средний'}], 'experience': [{'start': '01-03-2018', 'end': '01-09-2020', 'position': 'Программист C/С++', 'description': 'Обязанности: Разработка программ и библио

IndexError: ignored

In [ ]:
def skill_search(col, skills_arg):
  '''
  Функция для поиска ключевых слов в резюме
  '''
  for skill in skills_arg:
      df[skill] = 0
      skilled_ids = df[col].apply(lambda x: x if skill in str(x).lower() else np.nan).dropna().index
      df.loc[df.index.isin(skilled_ids), skill] = 1

In [ ]:
df.rename(columns = {0:'gender',1:'city',2:'wanted_spec', 3:'salary', 4:'education', 5:'experience',
                    6:'skill_set', 7:'skills', 8:'is_passed'}, inplace = True)


### Cleaning and conversion

In [ ]:
df['education'].value_counts()

Высшее образование                    198
Неоконченное высшее образование        51
Высшее образование (Магистр)           36
Среднее специальное образование        34
Высшее образование (Бакалавр)          34
Образование                            16
Higher education                        5
Среднее образование                     4
Higher education (bachelor)             4
Высшее образование (Кандидат наук)      3
Secondary special education             2
Education                               2
Secondary education                     1
Name: education, dtype: int64

In [ ]:
df.loc[df.education=='Higher education ', 'education'] = 'Высшее образование'

In [ ]:
df['gender'].value_counts()

Мужчина    347
Женщина     29
Male        12
Female       2
Name: gender, dtype: int64

In [ ]:
df.loc[df.gender=='Female', 'gender'] = 'Женщина'
df.loc[df.gender=='Male', 'gender'] = 'Мужчина'

In [ ]:
specs = list(df['wanted_spec'].value_counts().index)
for i in range(len(specs)):
    df.loc[df['wanted_spec']==specs[i], 'wanted_spec'] = i
    
cities = list(df['city'].value_counts().index)
for i in range(len(cities)):
    df.loc[df['city']==cities[i], 'city'] = i   
    
edu = list(df['education'].value_counts().index)
for i in range(len(edu)):
    df.loc[df['education']==edu[i], 'education'] = i  

In [ ]:
df = df.join(pd.get_dummies(df['gender']))
df.drop('gender', axis = 1, inplace = True)

In [ ]:
df.head(1)

,city,wanted_spec,salary,education,experience,skill_set,skills,Женщина,Мужчина
0,0,146,120000,2,[Программист C/С++],"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,0,1


Чем заполнить пропуски в данных?

In [ ]:
df['salary'].fillna(0, inplace = True)

In [ ]:
#none_salary = df.loc[df.salary == 0]
#df = df.loc[df.salary != 0]

Заполним пропуски в зарплатах средними значениями по каждой специализации

In [ ]:
#none_salary_spec = list(none_salary['wanted_spec'].value_counts().index)

#for spec in none_salary_spec:
#    mean_spec_salary = df.loc[df.wanted_spec == spec]['salary'].mean()
#    none_salary.loc[none_salary.wanted_spec == spec, 'salary'] = mean_spec_salary

Удалим строки, по которым не удалось найти зарплаты

In [ ]:
#none_salary.dropna(inplace = True)

In [ ]:
#df = pd.concat([df, none_salary])

In [ ]:
skills_for_candidates = ['английский']
col = 'skills'
skill_search(col, skills_for_candidates)

In [ ]:
skill_set_for_candidates = ['машинное обучение', 'sql', 'data mining', 'data analysis']
col = 'skill_set'
skill_search(col, skill_set_for_candidates)

In [ ]:
exp_for_candidates = ['программист c', '++', 'react']
col = 'experience'
skill_search(col, exp_for_candidates)

In [ ]:
count = ['experience', 'skill_set', 'skills']
df.drop(count, axis=1, inplace=True)

In [ ]:
df.head()

,city,wanted_spec,salary,education,Женщина,Мужчина,английский,машинное обучение,sql,data mining,data analysis,программист c,++,react
0,0,146,120000,2,0,1,0,1,1,1,1,1,1,0
1,0,229,0,0,0,1,0,0,0,0,0,0,0,1
2,1,233,0,0,0,1,0,0,1,0,0,0,0,0
3,1,25,0,7,0,1,0,0,1,0,0,0,0,0
4,1,209,30000,0,1,0,0,0,1,0,0,0,0,0


## Machine learning

Модель, которая будет обучаться на резюме действующих кандидатов.

Рекомендательная система будет чаще рекомендовать тех кандидатов, кто похож на успешных действующих сотрудников компании

In [ ]:
X = #требуются резюме действующих сотрудников
y =  #требуются резюме действующих сотрудников
X.drop('is_passed', axis=1, inplace=True)

In [ ]:
X.head()

,city,wanted_spec,salary,education,Женщина,Мужчина,английский,машинное обучение,sql,data mining,data analysis,программист c_experience,++_experience,react_experience
0,0,49,120000,2,0,1,0,1,1,1,1,1,1,0
1,0,194,0,0,0,1,0,0,0,0,0,0,0,1
2,1,41,0,0,0,1,0,0,1,0,0,0,0,0
3,1,31,0,8,0,1,0,0,1,0,0,0,0,0
4,1,113,30000,0,1,0,0,0,1,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
proba = pd.DataFrame(rfc.predict_proba(X_test))[1]

accuracy_score(rfc.predict(X_test), y_test)

0.6794871794871795

In [ ]:
proba.index = X_test.index
X_test['proba'] = proba

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
